In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.2
seed = 0
alpha = 1e-2
lamda = 1e-3
eta = 1e-2
d0 = 2
no_users = sum(cluster_sizes)
batch_size = 20
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.11111111 0.         0.         0.11111111
  0.11111111 0.         0.11111111 0.11111111 0.         0.11111111
  0.         0.11111111 0.11111111 0.         0.         0.
  0.         0.        ]
 [0.         0.         0.14285714 0.         0.125      0.
  0.         0.         0.14285714 0.         0.         0.
  0.         0.         0.125      0.         0.         0.1
  0.         0.14285714]
 [0.11111111 0.14285714 0.         0.16666667 0.         0.
  0.11111111 0.16666667 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.16666667 0.         0.         0.
  0.11111111 0.16666667 0.         0.         0.         0.
  0.         0.         0.         0.16666667 0.         0.
  0.         0.        ]
 [0.         0.125      0.         0.         0.         0.
  0.11111111 0.125      0.         0.         0.         0.
  0.125      0.125      0.         0.      

In [4]:
def load_dataset():
    transforms_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
    mnist_data_train = datasets.MNIST('./data/mnist', train=True, download=True, transform=transforms_mnist)
    mnist_data_test = datasets.MNIST('./data/mnist', train=False, download=True, transform=transforms_mnist)

    return mnist_data_train, mnist_data_test

In [5]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [6]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0])
W2 = np.array([-2.0, 2.0])
W = [W1, W2]
m = 200
n = 2
noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i]) + np.random.normal(0,noise_sd)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [7]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [8]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(2, 1, bias=False)
        #self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [9]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [10]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(9.0082, grad_fn=<MseLossBackward0>) tensor([ 3.0721, -5.1246]) tensor([-0.5398, -0.6402], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(8.5512, grad_fn=<MseLossBackward0>) tensor([ 2.2079, -5.3870]) tensor([-0.5705, -0.5889], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(6.8946, grad_fn=<MseLossBackward0>) tensor([ 2.1755, -4.2318]) tensor([-0.5926, -0.5351], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(6.2778, grad_fn=<MseLossBackward0>) tensor([ 1.9274, -3.9655]) tensor([-0.6143, -0.4928], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(7.8044, grad_fn=<MseLossBackward0>) tensor([ 2.8675, -4.7654]) tensor([-0.6336, -0.4531], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(7.4016, grad_fn=<MseLossBackward0>) tensor([ 2.0697, -5.0032]) tensor([-0.6623, -0.4054], grad_fn=<CatBackward0>)
torch.Size([50, 1])
to

20 tensor(0.4800, grad_fn=<MseLossBackward0>) tensor([ 0.6002, -1.2306]) tensor([-1.6230,  1.4038], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
20 tensor(0.3912, grad_fn=<MseLossBackward0>) tensor([ 0.5872, -0.9670]) tensor([-1.6290,  1.4161], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
20 tensor(0.3555, grad_fn=<MseLossBackward0>) tensor([ 0.5313, -0.9005]) tensor([-1.6349,  1.4258], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
21 tensor(0.4455, grad_fn=<MseLossBackward0>) tensor([ 0.7220, -1.1167]) tensor([-1.6402,  1.4348], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
21 tensor(0.4158, grad_fn=<MseLossBackward0>) tensor([ 0.5621, -1.1431]) tensor([-1.6474,  1.4459], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
21 tensor(0.3390, grad_fn=<MseLossBackward0>) tensor([ 0.5497, -0.8983]) tensor([-1.6531,  1.4574], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
21 tensor(0.3081

43 tensor(0.0193, grad_fn=<MseLossBackward0>) tensor([ 0.1581, -0.2275]) tensor([-1.9180,  1.8870], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
43 tensor(0.0178, grad_fn=<MseLossBackward0>) tensor([ 0.1315, -0.2263]) tensor([-1.9196,  1.8893], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
43 tensor(0.0147, grad_fn=<MseLossBackward0>) tensor([ 0.1276, -0.1779]) tensor([-1.9209,  1.8915], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
43 tensor(0.0133, grad_fn=<MseLossBackward0>) tensor([ 0.1177, -0.1644]) tensor([-1.9222,  1.8933], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
44 tensor(0.0168, grad_fn=<MseLossBackward0>) tensor([ 0.1475, -0.2117]) tensor([-1.9234,  1.8950], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
44 tensor(0.0154, grad_fn=<MseLossBackward0>) tensor([ 0.1230, -0.2102]) tensor([-1.9249,  1.8971], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
44 tensor(0.0127

65 tensor(0.0009, grad_fn=<MseLossBackward0>) tensor([ 0.0346, -0.0467]) tensor([-1.9815,  1.9773], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
65 tensor(0.0008, grad_fn=<MseLossBackward0>) tensor([ 0.0303, -0.0449]) tensor([-1.9818,  1.9778], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
65 tensor(0.0006, grad_fn=<MseLossBackward0>) tensor([ 0.0291, -0.0354]) tensor([-1.9821,  1.9782], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
65 tensor(0.0006, grad_fn=<MseLossBackward0>) tensor([ 0.0274, -0.0324]) tensor([-1.9824,  1.9786], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
66 tensor(0.0007, grad_fn=<MseLossBackward0>) tensor([ 0.0323, -0.0435]) tensor([-1.9827,  1.9789], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
66 tensor(0.0007, grad_fn=<MseLossBackward0>) tensor([ 0.0284, -0.0417]) tensor([-1.9830,  1.9793], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
66 tensor(0.0006

79 tensor(8.7988e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0112, -0.0127]) tensor([-1.9931,  1.9921], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
79 tensor(8.1815e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0108, -0.0116]) tensor([-1.9932,  1.9923], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
80 tensor(0.0001, grad_fn=<MseLossBackward0>) tensor([ 0.0123, -0.0160]) tensor([-1.9933,  1.9924], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
80 tensor(9.2081e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0111, -0.0149]) tensor([-1.9934,  1.9925], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
80 tensor(7.6334e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0104, -0.0118]) tensor([-1.9936,  1.9927], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
80 tensor(7.1110e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0101, -0.0107]) tensor([-1.9937,  1.9928], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 

93 tensor(1.1773e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0043, -0.0041]) tensor([-1.9974,  1.9972], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
94 tensor(1.5093e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0047, -0.0060]) tensor([-1.9974,  1.9972], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
94 tensor(1.2929e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0044, -0.0053]) tensor([-1.9975,  1.9973], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
94 tensor(1.0536e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0039, -0.0042]) tensor([-1.9975,  1.9974], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
94 tensor(1.0286e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0040, -0.0038]) tensor([-1.9976,  1.9974], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
95 tensor(1.3203e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0044, -0.0056]) tensor([-1.9976,  1.9974], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([

In [11]:
parameters_to_vector(model.parameters())

tensor([-1.9983,  1.9982], grad_fn=<CatBackward0>)

In [12]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [13]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[j][model.user_id] - projected_weights[model.user_id][j]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [14]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = d0, parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[i].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[i][j], parameters_to_vector(models[i].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [15]:
print(projection_list[0])

[0, 0, tensor([[0.8738, 0.0000],
        [0.0000, 0.8738]]), 0, 0, tensor([[3.5087, 0.0000],
        [0.0000, 3.5087]]), 0, 0, tensor([[0.7108, 0.0000],
        [0.0000, 0.7108]]), tensor([[1.2104, 0.0000],
        [0.0000, 1.2104]]), 0, 0, 0, 0, 0, 0, 0, 0, tensor([[2.0163, 0.0000],
        [0.0000, 2.0163]]), 0]


In [16]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [17]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [18]:
torch.norm(datapoints[model.user_id]['exact_weights'])

tensor(2.8284, dtype=torch.float64)

In [19]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([-0.4875, -0.6330])


tensor([-0.9749, -1.2660], grad_fn=<CatBackward0>)

In [20]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            mat_vec_sum = torch.zeros(d0)
            for k in G.neighbors(i):
                mat_vec_sum = torch.add(mat_vec_sum, projected_weights[i][k] - projected_weights[k][i])
            temp_mat = torch.outer(mat_vec_sum, weights).clone()


            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * temp_mat)
                                         
    projected_weights = []                                          
    update_ProjWeight(projection_list, projected_weights, first_run=False)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Relative Error %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 2/2000 [00:00<04:23,  7.57it/s]

Training_loss 8.40115,   Relative Error 1.09328
Training_loss 8.20239,   Relative Error 1.08036
Training_loss 7.90893,   Relative Error 1.06084


  0%|          | 6/2000 [00:00<03:17, 10.10it/s]

Training_loss 7.68636,   Relative Error 1.04565
Training_loss 7.44266,   Relative Error 1.02899
Training_loss 7.13862,   Relative Error 1.00783


  0%|          | 8/2000 [00:00<03:29,  9.51it/s]

...

Training_loss 0.00000,   Relative Error 0.00019
Training_loss 0.00000,   Relative Error 0.00019


100%|█████████▉| 1998/2000 [05:24<00:00,  6.72it/s]

Training_loss 0.00000,   Relative Error 0.00019
Training_loss 0.00000,   Relative Error 0.00019


100%|██████████| 2000/2000 [05:24<00:00,  6.16it/s]

Training_loss 0.00000,   Relative Error 0.00019
Training_loss 0.00000,   Relative Error 0.00019


In [21]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([-2.0005,  2.0000], grad_fn=<CatBackward0>)

In [22]:
for j in G.neighbors(0):
    print(j)

2
5
8
9
18


In [23]:
parameters_to_vector(models[0].parameters())

tensor([2.0099, 2.0057], grad_fn=<CatBackward0>)

In [24]:
projection_list[0]

[0,
 0,
 tensor([[ 0.3123, -0.5603],
         [-0.3539,  0.5207]], grad_fn=<AddBackward0>),
 0,
 0,
 tensor([[ 2.9472, -0.5603],
         [-0.3539,  3.1555]], grad_fn=<AddBackward0>),
 0,
 0,
 tensor([[ 0.1492, -0.5603],
         [-0.3539,  0.3576]], grad_fn=<AddBackward0>),
 tensor([[ 0.6489, -0.5603],
         [-0.3539,  0.8573]], grad_fn=<AddBackward0>),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 tensor([[ 1.4548, -0.5603],
         [-0.3539,  1.6631]], grad_fn=<AddBackward0>),
 0]

In [25]:
projected_weights[0]

[0,
 0,
 tensor([-0.4961,  0.3330]),
 0,
 0,
 tensor([4.7998, 5.6177]),
 0,
 0,
 tensor([-0.8238,  0.0060]),
 tensor([0.1805, 1.0081]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 tensor([1.8002, 2.6244]),
 0]

In [26]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [27]:
print(test_loss)

[8.40115285e+00 8.20238960e+00 7.90893400e+00 ... 6.67708662e-07
 6.64972248e-07 6.64203128e-07]


In [28]:
np.save( 'training_loss_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), test_loss)
np.save('relative_error_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), total_rel_error)

In [29]:
'training_loss_sheave_fml' + str(eta).replace('.', '_')

'training_loss_sheave_fml0_01'

In [30]:
'relative_error_sheave_fml' + str(eta).replace('.', '_')

'relative_error_sheave_fml0_01'